<a href="https://colab.research.google.com/github/adititadkod15-tech/HinglishLID1/blob/main/l3cube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
from transformers import pipeline


In [17]:
#  1. Load the model
model_id = "l3cube-pune/hing-bert-lid"
lid_pipeline = pipeline(
    "token-classification",
    model=model_id,
    device=0 # Uses cuda:0
)

Device set to use cuda:0


In [18]:
def get_word_lists_fixed(sentence):
    # Manual reconstruction to handle sub-word tokens like '##'
    results = lid_pipeline(sentence)

    hindi_words = []
    english_words = []

    current_word = ""
    current_label = ""

    for entity in results:
        raw_word = entity['word']
        label = entity['entity'] # Uses 'HI' and 'EN' as per your debug

        if raw_word.startswith("##"):
            current_word += raw_word.replace("##", "")
        else:
            if current_word:
                if current_label == 'HI':
                    hindi_words.append(current_word)
                elif current_label == 'EN':
                    english_words.append(current_word)

            current_word = raw_word
            current_label = label

    # Save the final word
    if current_word:
        if current_label == 'HI':
            hindi_words.append(current_word)
        elif current_label == 'EN':
            english_words.append(current_word)

    return hindi_words, english_words

In [20]:
# --- Interactive User Input Loop ---
sample_sentences =[
    'Pahle toh hum baat karte hain Cambodian Civil War ki (999 words). Wahan par jo hua, woh sach mein heartache aur shok se bhara tha. Lakhon logon ki zindagiyan barbad ho gayihi. Kitni hi auraten vidhvaa ho gayihi, aur kitne bachche anath! Sab ke chehron par sirf nirasha aur melancholic hi nazar aata hoga us samay.',
    'Mujhe yaad hai jab maine ek documentary dekha tha is war ke baare mein. Woh sab sunkar mera rooh kaanp gaya tha. Bachchon ko apne maa-baap ko khote dekhna, auraton ki vidhvavastha, ye sab dekhkar ekdum se mera man udaas aur dismay se bhar gaya tha. ',
    'Jaisa ki aap sab jaante hain, wars ke beech chemical aur biological weapons ka istemaal bhi hota hai. Inse toh aur bhi nuksan hota hai zindagiyon ka. Ek missile ya bomb se lakhon log dukhi ho jaate hain - apne ghar gawaa dete hain, apne parivaar ko khaa dete hain. Ye sab sochna hi itna hidaayatheen karta hai',
    'Aaj ki is roundtable discussion ka mukhy vishay bahut hi bhavuk hai (999 words) - wars aur conflicts ke emotional asar par charcha karni hai. Ye ek aisi cheez hai jo har insaan ko chhoo kar jaati hai, chahe woh usmein directly shaamil ho ya na ho.',
    'Fijian Coups ke baare mein padha tha maine kuch dino pehle. Wahan par bhi bahut kuchh hua tha. Aakhir kyun hoti hain ye sab cheezein? Kyun koi use bombs aur weapons karta hai doosre ki zindagiyon ko barbad karne ke liye? Ye sab jaanne ke baad toh main ekdum hi nimhans ho gaya tha.',
    'Aur jaisa ki hum jaante hain, India bhi aise hi kayi wars aur insurgencies se guzra hai. Jaise ki Naxalite-Maoist insurgency, aur Eritrean War of Independence. In dono mein kitne hi jawaan shaheed hue, aur kitne gareeb logon ko apni zindagiyaan gawani padin. Sab ke ghar barbad ho gaye, sari umeeden toot gayihi.',
    'War politics ke peeche badi wajah ye fear of war hota hai har country ka (999 words). Har koi chahta hai ki unhe jeetna hai, unhe power mein rehna hai. Issi liye ye sab wars shuru hote rehte hain. Par in sab ke peeche, sabse bada nuksan toh aam insaanon ka hi hota hai. '
]

sample_sentences = [
    'hello neevu yen madlikatriki nanu iga malkotini light off madri i am tired '
]

print("--- Hinglish Language Analyzer Ready ---")
print("Type 'exit' or 'quit' to stop.")


for sentence in sample_sentences:
      print(sentence)
      if sentence.lower() in ['exit', 'quit']:
          print("Exiting...")
          break

      if not sentence.strip():
          continue

      hi, en = get_word_lists_fixed(sentence)

      print("-" * 30)
      print(f"Hindi Words:   {hi}")
      print(f"English Words: {en}")
      print("-" * 30)

--- Hinglish Language Analyzer Ready ---
Type 'exit' or 'quit' to stop.
hello neevu yen madlikatriki nanu iga malkotini light off madri i am tired 
------------------------------
Hindi Words:   ['neevu', 'madlikatriki', 'nanu', 'malkotini', 'madri']
English Words: ['hello', 'yen', 'iga', 'light', 'off', 'i', 'am', 'tired']
------------------------------


### **TRANSLITERATION**

In [8]:
!pip install indic-transliteration

  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [indic-transliteration]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
urduhack 1.0.3 requires Click~=7.1, but you have click 8.3.1 which is incompatible.


In [14]:
import torch
from transformers import pipeline
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
import re

# 1. Initialize your working LID model
lid_pipeline = pipeline("token-classification", model="l3cube-pune/hing-bert-lid", device=0)

def process_sentence_to_hindi(sentence):
    # Detect language labels (HI, EN, etc.)
    results = lid_pipeline(sentence)

    # Reconstruct words from sub-tokens
    words_data = []
    current_word, current_label = "", ""
    for entity in results:
        raw_word = entity['word']
        label = entity['entity']
        if raw_word.startswith("##"):
            current_word += raw_word.replace("##", "")
        else:
            if current_word: words_data.append((current_word, current_label))
            current_word, current_label = raw_word, label
    if current_word: words_data.append((current_word, current_label))

    # 2. Extract Hindi words and convert to Devanagari
    final_output = []
    for word, label in words_data:
        if label == 'HI':
            # Clean non-alphabet characters
            clean_word = re.sub(r'[^a-zA-Z]', '', word).lower()

            if len(clean_word) > 0:
                # Transliterate Roman to Devanagari
                # We use ITRANS scheme for better accuracy with common Roman Hindi
                hindi_script = transliterate(clean_word, sanscript.ITRANS, sanscript.DEVANAGARI)
                final_output.append((clean_word, hindi_script))

    return final_output

# --- TEST ---
sentence = "Pahle toh hum baat karte hain Cambodian Civil War ki (999 words). Wahan par jo hua, woh sach mein heartache aur shok se bhara tha. Lakhon logon ki zindagiyan barbad ho gayihi. Kitni hi auraten vidhvaa ho gayihi, aur kitne bachche anath! Sab ke chehron par sirf nirasha aur melancholic hi nazar aata hoga us samay."
results = process_sentence_to_hindi(sentence)


for roman, script in results:
    print(f"roman:{roman:<15} hindi:{script}")

Device set to use cuda:0


roman:pahle           hindi:पह्ले
roman:toh             hindi:तोह्
roman:hum             hindi:हुम्
roman:baat            hindi:बात्
roman:karte           hindi:कर्ते
roman:hain            hindi:हैन्
roman:ki              hindi:कि
roman:wahan           hindi:वहन्
roman:par             hindi:पर्
roman:jo              hindi:जो
roman:hua             hindi:हुअ
roman:woh             hindi:वोह्
roman:sach            hindi:सच्
roman:mein            hindi:मेइन्
roman:aur             hindi:और्
roman:shok            hindi:शोक्
roman:se              hindi:से
roman:bhara           hindi:भर
roman:lakhon          hindi:लखोन्
roman:logon           hindi:लोगोन्
roman:ki              hindi:कि
roman:zindagiyan      hindi:ज़िन्दगियन्
roman:barbad          hindi:बर्बद्
roman:ho              hindi:हो
roman:gayihi          hindi:गयिहि
roman:kitni           hindi:कित्नि
roman:hi              hindi:हि
roman:auraten         hindi:औरतेन्
roman:vidhvaa         hindi:विध्वा
roman:ho              hindi:हो
roman:gay